In [ ]:
import sys

sys.path.append("..")
from omegaconf import OmegaConf
from sgm.util import instantiate_from_config

config = OmegaConf.load(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/configs/example_training/svd_interpolation.yaml"
)
print(config)
config["model"]["params"]["ckpt_path"] = (
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/checkpoints/svd.safetensors"
)
config["model"]["params"]["network_config"]["params"]["fine_tuning_method"] = None
model = instantiate_from_config(config.model)

In [ ]:
for name, p in model.named_parameters():
    if "time_" in name:
        print(name, p.shape)

In [ ]:
from safetensors.torch import load_file

model_ckpt = load_file("../checkpoints/svd.safetensors")

print(model_ckpt.keys())

In [ ]:
unet = model.model.diffusion_model

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import apply_lora
import torch

# activate_lora = add_lora_to(unet, ["SpatialVideoTransformer"], search_class=[torch.nn.Linear], r=32, dropout=0, lora_bias="none")
apply_lora(unet, filters=[".input_blocks"], rank=16, all_modules_in_filter=True)

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import LoRA

lora = {}
lora["weights"] = torch.nn.ModuleList()
for module in unet.modules():
    if isinstance(module, LoRA) or (
        hasattr(module, "_fsdp_wrapped_module") and isinstance(module._fsdp_wrapped_module, LoRA)
    ):
        print(module.requires_grad_())
        lora["weights"].append(module)

In [ ]:
lora

In [ ]:
for param in unet.parameters():
    if param.requires_grad:
        print("requires grad")
    else:
        print("does not require grad")

In [ ]:
for p in unet.input_blocks[0].parameters():
    print(p)

In [ ]:
for name, module in unet.named_modules():
    print(name)

In [ ]:
from sgm.modules.diffusionmodules.adapters.lora import get_module_names

module_names = get_module_names(unet, filters=[".transformer_blocks"], all_modules_in_filter=True)

In [ ]:
from peft import LoraModel, LoraConfig

lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=module_names,
)
lora_unet = LoraModel(unet, lora_config, "bite")

In [ ]:
lora_unet.input_blocks[4].disable_adapter()